<!--NOTEBOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="./figures/LogoOpenclassrooms.png">
<font size="4">
    
Cette étude a été réalisée dans le cadre du 6ème projet de ma formation Datascientist dispensée en MOOC par 

<font color='blus'>Openclassrooms / écoles Centrale-Supélec</font>.




<p></p><p></p><p></p>





**Le problème posé :**

# <font color='blus'>Indexation d'images</font>

Vous êtes bénévole pour l'association de protection des animaux de votre quartier. C'est d'ailleurs ainsi que vous avez trouvé votre compagnon idéal, Snooky. Du coup, vous vous demandez ce que vous pouvez faire en retour pour aider l'association.

Vous apprenez, en discutant avec un bénévole, que leur base de données de pensionnaires commence à s'agrandir et qu'ils n'ont pas toujours le temps de référencer les images des animaux qu'ils ont accumulées depuis plusieurs années. Ils aimeraient donc réaliser un index de l’ensemble de la base de données d’images qu’ils possèdent, pour classer les chiens par races.

**<font color='blus'>Les données</font>**

Les bénévoles de l'association n'ont pas eu le temps de réunir les différentes images des pensionnaires dispersées sur leurs disques durs. Pas de problème, vous développerez un algorithme en utilisant le Stanford Dogs Dataset pour entraîner votre algorithme.

**<font color='blus'>Votre mission</font>**

En tant que Data Scientist, l'association vous demande de réaliser un algorithme de détection de la race du chien sur une photo, afin d'accélérer leur travail d’indexation.

**<font color='blus'>Contraintes</font>**

Lors de ce projet, vous mettrez en œuvre deux approches.

* Une approche classique : il s’agit de pre-processer des images avec des techniques spécifiques (e.g.whitening, equalisation, filtre linéaire/laplacien/gaussien, éventuellement modifier la taille des images), puis d’extraire des features (e.g. texture, corners, edges et SIFT detector). Il faut ensuite réduire les dimensions, soit par des approches classiques (e.g. PCA, k-means) soit avec une approche par histogrammes et dictionary learning (bag-of-words appliqué aux images), puis appliquer des algorithmes de classification standards.



* Lors de l’analyse exploratoire, vous regarderez si les features extraites et utilisées en classification sont prometteuses en utilisant des méthodes de réduction de dimension pour visualiser le dataset en 2D. Cela vous permettra d’affiner votre intuition sur les différents traitements possibles, sans que cela ne se substitue à des mesures de performances rigoureuses.



* Une approche s’appuyant sur l’état de l’art et l’utilisation de CNN (réseaux de neurones convolutionnels). Compte tenu de la taille et de la complexité du dataset, et de la puissance de calcul à votre disposition, il est très difficile d’obtenir de bonnes performances (pour ça, essayez MNIST). Aussi, est-il recommandé d’utiliser le transfer learning, c’est-à-dire utiliser un réseau déjà entraîné, et le modifier pour répondre à votre problème. Une première chose obligatoire est de ré-entraîner les dernières couches pour prédire les classes qui vous intéressent seulement. Il est également possible d’adapter la structure (supprimer certaines couches par exemple) ou de ré-entraîner le modèle avec un très faible learning rate pour ajuster les poids à votre problème (plus long) et optimiser les performances.

In [1]:
from IPython.display import display, clear_output


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import P7_DataBreed
import p5_util

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

from  sklearn import model_selection
import numpy as np


Using TensorFlow backend.
/home/bangui/.local/lib/python3.6/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


# <font color='blus'>1. Data is loaded</font>

In [4]:
import p5_util
filename='./data/arr_keras_X_y_train_test.dump'
(X_train, X_test, y_train, y_test) = p5_util.object_load(filename)

p5_util.object_load : fileName= ./data/arr_keras_X_y_train_test.dump


In [5]:
X_train.shape, X_test.shape

((414, 224, 224, 3), (47, 224, 224, 3))

# <font color='blus'>2. Breeds classification using CNN VGG16 network</font>

### Building VGG16 Keras layers

#### 1st convolution layer is added 

In [4]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Flatten, Dense, Dropout

VGG16Seq = Sequential()  # Création d'un réseau de neurones vide 

# Ajout de la première couche de convolution, suivie d'une couche ReLU
w=X_train.shape[1]
h=X_train.shape[2]
c=X_train.shape[3]
print(w,h,c)
VGG16Seq.add(Conv2D(64, (3, 3), input_shape=(w, h, c), padding='same', activation='relu'))
VGG16Seq.add(Conv2D(64, (3, 3), activation='relu'))
VGG16Seq.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

224 224 3


#### 2nd convolution layer is added 

In [5]:
# Ajout de la deuxième couche de convolution, suivie  d'une couche ReLU
VGG16Seq.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
VGG16Seq.add(Conv2D(64, (3, 3), activation='relu'))
VGG16Seq.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

#### 3th convolution layer is added 

In [6]:
# Ajout de la deuxième couche de convolution, suivie  d'une couche ReLU
VGG16Seq.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
VGG16Seq.add(Conv2D(64, (3, 3), activation='relu'))
VGG16Seq.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

#### Pooling layer is added 

In [8]:
nb_labels = y_train.shape[0]
print("Number of labels for training process : "+str(nb_labels))
VGG16Seq.add(Flatten())  # Conversion des matrices 3D en vecteur 1D

Number of labels for training process : 414


#### 1st Dense layer is added

In [9]:
VGG16Seq.add(Dense(nb_labels, activation='relu'))
VGG16Seq.add(Dropout(0.5))

#### 2nd Dense layer is added

#### 3th Dense layer is added

In [10]:
import numpy as np
# Ajout de la dernière couche fully-connected qui permet de classifier
VGG16Seq.add(Dense(nb_labels, activation='softmax'))

#### VGG16 compilation

In [11]:
from keras import optimizers
sgd = optimizers.SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
rmsprop = optimizers.RMSprop(lr=1e-4)

#my_VGG16.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
#my_VGG16.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
VGG16Seq.compile(optimizer=rmsprop, loss='sparse_categorical_crossentropy', metrics=['accuracy'])


#### Keras Sequential model is dumped

In [12]:
import p5_util
p5_util.object_dump(VGG16Seq,'./data/VGG16Seq_sgd.dump')

VGG16Seq.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 222, 222, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 111, 111, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 111, 111, 64)      36928     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 109, 109, 64)      36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 54, 54, 64)        36928     
__________

#### Result is picked and saved into a file

In [13]:
import p5_util
filename = './data/dict_cls_cnn_built.dump'
dict_cls_cnn_built = {'CNN Built':0.75}
p5_util.object_dump(dict_cls_cnn_built,filename)

## Loading VGG166 Sequential model

In [ ]:
from IPython.display import display, clear_output


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import P7_DataBreed
import p5_util

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

from  sklearn import model_selection
import numpy as np


In [ ]:
import p5_util
if True:
    VGG16Seq = p5_util.object_load('./data/VGG16Seq_sgd.dump')
    VGG16Seq.summary()

import p5_util
import P7_DataBreed
if False :
    oP7_DataBreed = p5_util.object_load('./data/oP7_DataBreed.dump')
    oP7_DataBreed.show()
    oP7_DataBreed.breed_show()


## Test for images classification

#### Data is loaded form dumped file

In [14]:
import p5_util
filename='./data/arr_keras_X_y_train_test.dump'
(arr_X_train,arr_X_test, arr_y_train, arr_y_test) = p5_util.object_load(filename)
print(arr_X_train.shape,arr_X_test.shape,arr_y_train.shape,arr_y_test.shape)

#### Data normalization

arr_X_train = arr_X_train.astype('float32')
arr_X_test = arr_X_test.astype('float32')

# Scale the data to lie between 0 to 1
arr_X_train /= 255
arr_X_test /= 255

p5_util.object_load : fileName= ./data/arr_keras_X_y_train_test.dump
(414, 224, 224, 3) (47, 224, 224, 3) (414, 3) (47, 3)


#### Model training

In [15]:
print(arr_X_train.shape)
history = VGG16Seq.fit(arr_X_train, arr_y_train, epochs=50, batch_size=90,verbose=1,validation_data=(arr_X_test, arr_y_test))

(414, 224, 224, 3)


ValueError: Error when checking target: expected dense_2 to have shape (1,) but got array with shape (3,)

In [ ]:
[test_loss, test_acc] = VGG16Seq.evaluate(arr_X_test, arr_y_test)
print("Evaluation result on Test Data : Loss = {}, accuracy = {}".format(test_loss, test_acc))

In [ ]:
#Plot the Loss Curves
import matplotlib.pyplot as plt

plt.figure(figsize=[8,6])
plt.plot(history.history['loss'],'r',linewidth=1.0)
plt.plot(history.history['val_loss'],'b',linewidth=1.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Curves ',fontsize=16)
 
#Plot the Accuracy Curves
plt.figure(figsize=[8,6])
plt.plot(history.history['acc'],'r',linewidth=1.0)
plt.plot(history.history['val_acc'],'b',linewidth=1.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Accuracy Curves',fontsize=16)

In [ ]:
import p5_util
is_score_dumped=True
if is_score_dumped is True :
    filename = './data/dict_cls_score.dump'
    dict_cls_score = p5_util.object_load(filename)
else:
    dict_cls_score = dict()
dict_classifier = dict()

dict_cls_score

In [ ]:
y_pred = VGG16Seq.predict(arr_X_test)

In [ ]:
filename= './data/result_VGG16Seq.dump'
p5_util.object_dump(y_pred,filename)

#### Measure of accuracy

Result is stored into `arr_result` where :
* rows : number of observations from test dataset
* columns : probabibilty for each observation from train dataset.

For each row from `arr_result`, greater probability value is searched for. 
<br>
The result of search provides index of label into `list_index`.


In [ ]:
import p5_util
filename= './data/result_vgg16.dump'
p5_util.object_load(y_pred,filename)

In [ ]:
(loss, score_vgg16_seq) = VGG16Seq.evaluate(arr_X_test, arr_y_test, verbose=True)

score_vgg16_seq

In [ ]:
import pandas as pd
import p5_util
import p6_util_plot
is_dumped=True
if is_dumped is True:
    filename = './data/dict_cls_score.dump'
    dict_cls_score = p5_util.object_load(filename)
else :
    pass

dict_cls_score['VGG16 Seq'] = score_vgg16_seq
dict_benchmark_result = dict_cls_score.copy()

df_result = pd.DataFrame.from_dict( dict_benchmark_result, orient='index')
df_result.reset_index(inplace=True)
df_result.rename(columns={'index':'Classifier',0:'Score'}, inplace=True)
df_result
nb_images = oP7_DataBreed._sampling_breed_count*oP7_DataBreed._sampling_image_per_breed_count
nb_images = oP7_DataBreed.df_pil_image_kpdesc.shape[0]
if oP7_DataBreed.is_kp_filtered :
    title = "Benchmark classifiers accuracy / GMM clustering / "+str(nb_images)+" filtered splitted images / "+str(oP7_DataBreed.sampling_breed_count)+" breeds"
else :
    title = "Benchmark classifiers accuracy / GMM clustering / "+str(nb_images)+" splitted images / "+str(oP7_DataBreed.sampling_breed_count)+" breeds"

p6_util_plot.ser_item_occurency_plot(df_result.Classifier, df_result.Score*100, item_count=None, title=title,\
                                    p_reverse=False,p_x_title='Classifiers', p_y_title='Accuracy')

#### Classifier API
p5_util.object_dump(dict_cls_score,filename)

dict_cls_score

In [ ]:
oP7_DataBreed.show()
oP7_DataBreed.show_breed_name()